In [0]:
catalog_name = "rohitb_demo"
schema_name = "pdf_chat"
table_name = "parsed_pdf_docs"
full_table_path = f"{catalog_name}.{schema_name}.{table_name}"
chunked_table_path = f"{catalog_name}.{schema_name}.chunked_pdf_docs"
vector_index_path = f"{catalog_name}.{schema_name}.chunked_pdf_docs_index"

In [0]:
from databricks_langchain import DatabricksVectorSearch

vector_store = DatabricksVectorSearch(index_name=vector_index_path)
retriever = vector_store.as_retriever(search_kwargs={"k": 5})
retriever.invoke("total bill February 2024?")

In [0]:
%sql
select * from rohitb_demo.pdf_chat.pdf_chat_vector_search('Bill for the month of February 2024?')

# Examples of Tool Calling

## With VectorSearchRetrieverTool

In [0]:
from databricks_langchain import VectorSearchRetrieverTool, ChatDatabricks

# Initialize the retriever tool.
vs_tool = VectorSearchRetrieverTool(
  index_name=vector_index_path,
  tool_name="pdf_docs_retriever",
  tool_description="Retrieves billing information from historical billing documents." 
)
# Run a query against the vector search index locally for testing
vs_tool.invoke("total bill February 2024?")

In [0]:
# Bind the retriever tool to your Langchain LLM of choice
llm = ChatDatabricks(endpoint="databricks-meta-llama-3-1-70b-instruct")
llm_with_tools = llm.bind_tools([vs_tool])

# Chat with your LLM to test the tool calling functionality
llm_with_tools.invoke("total bill February 2024?")

## With UC Toolkit

In [0]:
from unitycatalog.ai.langchain.toolkit import UCFunctionToolkit

toolkit = UCFunctionToolkit(
    function_names=[
        "rohitb_demo.pdf_chat.pdf_chat_vector_search"
    ]
)
tools = toolkit.tools

In [0]:
from databricks_langchain import VectorSearchRetrieverTool, ChatDatabricks
# Bind the retriever tool to your Langchain LLM of choice
llm = ChatDatabricks(endpoint="databricks-meta-llama-3-1-70b-instruct")
llm_with_tools = llm.bind_tools(tools)

# Chat with your LLM to test the tool calling functionality
llm_with_tools.invoke("total bill February 2024?")